In [0]:
import collections
import datetime
import numpy as np
import pandas as pd
import subprocess
import tensorflow as tf
from tensorflow import keras
import time

In [2]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-05-27 00:58:18--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.146.197
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.146.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  77.9MB/s    in 20s     

2020-05-27 00:58:38 (77.3 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
from gensim.models import KeyedVectors

In [4]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
B = 32

In [6]:
subprocess.run(["rm", "-rf", "./logs/"])


CompletedProcess(args=['rm', '-rf', './logs/'], returncode=0)

In [0]:
def count_words(title_list):
    c = collections.Counter(
        word for title in title_list for word in title.split())
    return {word: i + 1 for i, (word, cnt) in enumerate(c.most_common()) if cnt > 1}


def label_encoder(title, word_dict):
    return [word_dict.get(word, 0) for word in title.split()]


In [0]:
train_df = pd.read_table('/content/drive/My Drive/Colab Notebooks/train.txt')
valid_df = pd.read_table('/content/drive/My Drive/Colab Notebooks/valid.txt')

In [0]:
word_dict = count_words(train_df['TITLE'])
num_words = len(word_dict) + 1  # word not in dict is labeled as 0


In [0]:
# label encode and padding with 0
train_X = keras.preprocessing.sequence.pad_sequences(
    train_df['TITLE'].map(lambda x: label_encoder(x, word_dict)))
valid_X = keras.preprocessing.sequence.pad_sequences(
    valid_df['TITLE'].map(lambda x: label_encoder(x, word_dict)))


In [0]:
train_y = train_df['CATEGORY'].map({'b': 0, 't': 1, 'e': 2, 'm': 3}).values
valid_y = valid_df['CATEGORY'].map({'b': 0, 't': 1, 'e': 2, 'm': 3}).values


In [0]:
def init_emb(word_dict, word2vec):
  emb = np.zeros((len(word_dict) + 1, 300))
  for k, v in word_dict.items():
    try:
      emb[v] = word2vec[k]
    except:
      pass
  return emb

In [0]:
emb = init_emb(word_dict, word2vec)

In [0]:
model = keras.Sequential([
    keras.layers.Embedding(num_words, 300, mask_zero=True, embeddings_initializer=tf.constant_initializer(
    value=emb
)),
    keras.layers.SimpleRNN(50),
    keras.layers.Dense(4, activation='relu')
])

In [0]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

In [0]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)


In [17]:
st = time.time()
model.fit(train_X, train_y, batch_size=B, epochs=10, validation_data=(valid_X, valid_y),
          callbacks=[tensorboard_callback])
ed = time.time()
print("elapsed_time: " + str(ed - st))

Epoch 1/10
334/334 [==============================] - 37s 110ms/step - loss: 0.8573 - accuracy: 0.6944 - val_loss: 0.6424 - val_accuracy: 0.7894
Epoch 2/10
334/334 [==============================] - 40s 118ms/step - loss: 0.6341 - accuracy: 0.7767 - val_loss: 0.5693 - val_accuracy: 0.8058
Epoch 3/10
334/334 [==============================] - 42s 125ms/step - loss: 0.5527 - accuracy: 0.7993 - val_loss: 0.5244 - val_accuracy: 0.8073
Epoch 4/10
334/334 [==============================] - 39s 116ms/step - loss: 0.4877 - accuracy: 0.8213 - val_loss: 0.4910 - val_accuracy: 0.8231
Epoch 5/10
334/334 [==============================] - 39s 116ms/step - loss: 0.4349 - accuracy: 0.8489 - val_loss: 0.4428 - val_accuracy: 0.8433
Epoch 6/10
334/334 [==============================] - 41s 122ms/step - loss: 0.3903 - accuracy: 0.8671 - val_loss: 0.4112 - val_accuracy: 0.8531
Epoch 7/10
334/334 [==============================] - 38s 113ms/step - loss: 0.3580 - accuracy: 0.8797 - val_loss: 0.4011 - val_ac